In [1]:
import pandas as pd 
import numpy as np

In [8]:
demand = pd.read_csv('demand.csv',index_col=0)

In [9]:
demand.head(2)

,1 Av_0,103 St - Corona Plaza_0,103 St_0,103 St_1,103 St_2,104 St_0,104 St_1,110 St_0,111 St_0,111 St_1,...,Whitlock Av_0,Wilson Av_0,Winthrop St_0,Woodhaven Blvd_0,Woodhaven Blvd_1,Woodlawn_0,Woodside - 61 St_0,World Trade Center_0,York St_0,Zerega Av_0
work_station,,,,,,,,,,,,,,,,,,,,,
1 Av_0,70.0,70.0,15.0,40.0,30.0,0.0,10.0,0.0,25.0,50.0,...,25.0,15.0,10.0,20.0,30.0,0.0,55.0,0.0,0.0,0.0
103 St - Corona Plaza_0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0


In [10]:
df = demand.iloc[:,0:1].reset_index()
df['from_master_id']=df.columns[1]
df = df.rename(columns={'work_station':'to_master_id',
                  df.columns[1]:'duration_diff'})

In [11]:
demand_0 = pd.DataFrame()

In [12]:
for i in range(len(demand.columns)):
#for i in range(5):
    df = demand.iloc[:,i:i+1].reset_index()
    df['from_master_id']=df.columns[1]
    df = df.rename(columns={'work_station':'to_master_id',
                  df.columns[1]:'demand'})
    
    demand_0 = pd.concat([demand_0,df])

In [13]:
demand_0 = demand_0.reset_index(drop=True)

In [14]:
demand_0 = demand_0[demand_0['to_master_id']!=demand_0['from_master_id']].reset_index(drop= True)

In [15]:
demand_0.to_csv('demand.csv.gz',compression='gzip')

In [4]:
demand_0 = pd.read_csv('demand.csv.gz',compression='gzip')

In [16]:
duration = pd.read_csv('remove_one_node_duration.csv.gz',compression='gzip')

In [43]:
del duration['Unnamed: 0']

In [35]:
### if none, means not reachable for 1 hour
duration = duration.fillna(3600)

In [46]:
duration_0 = duration.merge(demand_0,on = ['to_master_id','from_master_id'])

In [49]:
delay = {}
for station in duration_0.columns[2:-1]:
    delay[station]= sum(np.array(duration.loc[:,station]) *\
                        np.array(duration_0.loc[:,'demand']))
    

In [57]:
delay = pd.DataFrame(sorted(delay.items(), key=lambda x: x[1],reverse=True),columns=['station','delay'])

In [59]:
delay.to_csv('delay')